In [1]:
#dependencies 
#had to 
import pandas as pd 
import numpy as np 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
from coincallandwritepythonfile import refined_coin_list, injest_date, write_to_storage, convert_dataframe_to_file

In [2]:
#make sure no blank id values for query
refined_coin_list = refined_coin_list[refined_coin_list['id'] != ""]
#get ids for each coin and convert to list
coin_ids = refined_coin_list.id.to_list()

In [3]:
number_of_coin_ids = len(coin_ids)
#max number is 30 per minute for the API
number_of_queries_per_run = 25
rounded_up = np.ceil(number_of_coin_ids/ number_of_queries_per_run)
rounded_up

4.0

In [4]:
def Make_Call (id, api_key):
    url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
    parameters = {
        'id': id,
                }
    headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key
    }

    #make API call and return response as a dictionary
    session = Session()
    session.headers.update(headers)
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    #erroring on 'Data' sometimes, like a rogue id
    #erroring because can't make more than 30 calls per minute 
    return data

In [5]:

import time
  
#schedule.run_all(delay_seconds= 3).do(func)
def get_coin_info_breakup(list_of_coins_, api_key):
    """Function makes API call for each coin ID within list of coins.
    Designed to take list from top 100 coins and get their most recent price information. """

    #create an empty dataframe that we will append responses too
    df = pd.DataFrame()
    #a count variable to index the respones when trying to concat 
    #get parameters for API call
    number_of_coin_ids = len(list_of_coins_)
    #max number is 30 per minute for the API
    number_of_queries_per_run = 25
    rounded_up = np.ceil(number_of_coin_ids/ number_of_queries_per_run)
    coin_array = list_of_coins_
    test = np.array_split(coin_array,rounded_up)
    count =0
    #schedule.every(60).seconds.do(Make_Call(id, api_key))
    while count < rounded_up:
        for id in test[count]:
            try:
                data = Make_Call(id, api_key)
                cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')

                #expand nested quote column containing price information
                expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')     
                #set then index for count ie the number of runs 
                expanded_df.set_index(pd.Index([count]),inplace= True)

                #dropping nested columns that we won't explore during this exercise
                trial = cmc_df.drop(['quote','tags', 'platform'], axis= 1)
                
                #setting index as count variable so can join back
                trial.set_index(pd.Index([count]),inplace= True)
                
                #concat was tricky had to add
                #had to add key to get them to append properly
                coin_price_df = pd.concat([trial,expanded_df], axis=1, keys=['trial', 'expanded_df']  )
                #coin_price_df.reset_index(drop = True, inplace= True)
                coin_price_merge = pd.merge(trial, expanded_df)
                #this saves it as a coreframe

                    
                df = df.append(coin_price_merge) 
            except KeyError:
                continue
            
            #parse dict and load in relavant data to a DF
        time.sleep(60)
        count += 1
    return df 

In [6]:
df = get_coin_info_breakup(coin_ids,  config.coin_api_key)

In [7]:
def get_coin_info(list_of_coins_, api_key):
    """Function makes API call for each coin ID within list of coins.
    Designed to take list from top 100 coins and get their most recent price information. """

    #create an empty dataframe that we will append responses too
    df = pd.DataFrame()
    #a count variable to index the respones when trying to concat 
    count =0
    for id in list_of_coins_:
        url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
        parameters = {
            'id': id,
                    }
        headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
        }

        #make API call and return response as a dictionary
        try:
            session = Session()
            session.headers.update(headers)
            response = session.get(url, params=parameters)
            data = json.loads(response.text)
            #erroring on 'Data' sometimes, like a rogue id
            #erroring because can't make more than 30 calls per minute
        except KeyError:
            continue
        
        #parse dict and load in relavant data to a DF
        cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
         
        #expand nested quote column containing price information
        expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')     
        #set then index for count ie the number of runs 
        expanded_df.set_index(pd.Index([count]),inplace= True)

        #dropping nested columns that we won't explore during this exercise
        trial = cmc_df.drop(['quote','tags', 'platform'], axis= 1)
          
        #setting index as count variable so can join back
        trial.set_index(pd.Index([count]),inplace= True)
        
        #concat was tricky had to add
        #had to add key to get them to append properly
        coin_price_df = pd.concat([trial,expanded_df], axis=1, keys=['trial', 'expanded_df']  )
        #coin_price_df.reset_index(drop = True, inplace= True)
        coin_price_merge = pd.merge(trial, expanded_df)
        #this saves it as a coreframe

               
        df = df.append(coin_price_merge)        

        count +=1
        #df.set_index(pd.Index([count]),inplace= True)
    #had to drop level inorder to get ride of multilevel index   
    # #https://sparkbyexamples.com/pandas/pandas-multiindex-dataframe-examples/?msclkid=fba45bb6c10e11eca1d376f5b274bab7 
        
    #df = df.droplevel(0, axis =1)
    return df

        


In [8]:
#df = get_coin_info(coin_ids,  config.coin_api_key)

In [9]:
df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap
0,1,Bitcoin,BTC,bitcoin,9368,2013-04-28T00:00:00.000Z,21000000,1.902401e+07,1.902401e+07,1,...,6.0264,0.094334,2.009621,-5.263637,-17.779855,-0.041704,8.236571,7.449265e+11,41.4793,8.223006e+11
0,1027,Ethereum,ETH,ethereum,5664,2015-08-07T00:00:00.000Z,None,1.205521e+08,1.205521e+08,1,...,6.2479,0.008431,0.618778,-6.565916,-15.030114,3.264496,18.057848,3.466301e+11,19.2971,3.466301e+11
0,825,Tether,USDT,tether,32460,2015-02-25T00:00:00.000Z,None,8.317288e+10,8.571395e+10,1,...,1.0827,0.001782,0.004213,-0.002697,-0.044811,-0.029407,-0.022144,8.318808e+10,4.6315,8.572962e+10
0,1839,BNB,BNB,bnb,821,2017-07-25T00:00:00.000Z,165116760,1.632770e+08,1.632770e+08,1,...,0.7761,0.120806,0.268802,-6.862269,-10.814381,4.231141,6.252297,6.378295e+10,3.5523,6.450165e+10
0,3408,USD Coin,USDC,usd-coin,3822,2018-10-08T00:00:00.000Z,None,4.953288e+10,4.953288e+10,1,...,0.9168,0.038187,-0.018692,0.002826,0.054325,0.030348,-0.087030,4.953900e+10,2.7590,4.953900e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1896,0x,ZRX,0x,241,2017-08-16T00:00:00.000Z,1000000000,8.474961e+08,1.000000e+09,1,...,36.7627,1.825078,9.347844,19.039397,17.972704,59.291443,62.390519,7.331149e+08,0.0408,8.650363e+08
0,6836,Moonbeam,GLMR,moonbeam,38,2022-01-11T12:55:50.000Z,1000000000,2.302483e+08,1.016967e+09,1,...,-0.5394,0.074793,-4.312897,-26.338720,-5.267023,16.459499,-55.233867,6.953281e+08,0.0387,3.019905e+09
0,2586,Synthetix,SNX,synthetix-network-token,233,2018-03-14T00:00:00.000Z,212424133,1.148415e+08,2.152588e+08,1,...,-31.4878,2.286583,5.122610,-6.811221,4.959030,51.557076,26.560932,6.807514e+08,0.0379,1.259196e+09
0,2099,ICON,ICX,icon,80,2017-10-27T00:00:00.000Z,None,9.194831e+08,9.262104e+08,1,...,-20.5290,-0.087386,-0.662029,-6.070240,-26.349960,6.598463,3.770860,6.669202e+08,0.0371,6.717996e+08


In [10]:
#write to storage function takes dataset input as a CSV so had to convert
df["File_Name"] = "Coin_Price_Fact"
#Add column for injested at 
df["injest_datetime"] = injest_date
#convert to CSV to write to storage
df_csv = convert_dataframe_to_file(df, "csv")

In [11]:
#write the new DF to storage
storage_account_name = config.storage_account_name
storage_account_key = config.storage_account_key
container_name = "coinprice"
directory_name = "raw"

dataframe = write_to_storage(storage_account_name=storage_account_name, storage_account_key= storage_account_key, container_name= container_name, directory_name= directory_name,\
    dataset = df_csv, file_name= "coinprice.csv")

### Refine dataset to write to refined ADLS

In [ ]:
dataframe = write_to_storage(storage_account_name=storage_account_name, storage_account_key= storage_account_key,\
     container_name= container_name, directory_name= directory_name,\
    dataset = df_csv, file_name= "coinprice.csv")

In [14]:
#remove null values
dataframe = dataframe[dataframe.name.notnull()]
price_and_supply_columns = ['id', 'name', 'symbol','price', 'max_supply', 'circulating_supply', \
   'total_supply', 'market_cap','File_Name', 'injest_datetime']
volume_and_percent_change_columns = ['id', 'volume_24h', 'volume_change_24h', 'percent_change_1h', 'percent_change_24h',
       'percent_change_7d', 'percent_change_30d', 'percent_change_60d',
       'percent_change_90d','File_Name', 'injest_datetime']


df_price_and_supply = dataframe[price_and_supply_columns]
df_price_and_supply_csv= convert_dataframe_to_file(df_price_and_supply, "csv")

df_volume_and_percent_change = dataframe[volume_and_percent_change_columns]
df_volume_and_percent_change_csv = convert_dataframe_to_file(df_volume_and_percent_change, "csv")


,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap
1,1,Bitcoin,BTC,bitcoin,9368,2013-04-28T00:00:00.000Z,21000000,19024012.0,19024012.0,1,...,6.0264,0.09433416,2.00962139,-5.26363671,-17.77985497,-0.04170401,8.23657087,744926493487.7192,41.4793,822300593757.1
2,1027,Ethereum,ETH,ethereum,5664,2015-08-07T00:00:00.000Z,,120552114.9365,120552114.9365,1,...,6.2479,0.00843098,0.61877849,-6.56591623,-15.03011442,3.26449635,18.05784786,346630086126.4368,19.2971,346630086126.44
3,825,Tether,USDT,tether,32460,2015-02-25T00:00:00.000Z,,83172877108.45573,85713951815.53362,1,...,1.0827,0.00178176,0.00421294,-0.00269707,-0.04481057,-0.02940676,-0.02214403,83188083989.06807,4.6315,85729623292.55
4,1839,BNB,BNB,bnb,821,2017-07-25T00:00:00.000Z,165116760,163276974.63,163276974.63,1,...,0.7761,0.12080626,0.26880153,-6.86226944,-10.81438139,4.23114095,6.25229654,63782948407.62078,3.5523,64501647021.44
5,3408,USD Coin,USDC,usd-coin,3822,2018-10-08T00:00:00.000Z,,49532884985.03943,49532884985.03943,1,...,0.9168,0.0381869,-0.01869242,0.00282646,0.0543248,0.03034803,-0.08702955,49539003403.076096,2.759,49539003403.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1896,0x,ZRX,0x,241,2017-08-16T00:00:00.000Z,1000000000,847496054.8027713,1000000000.0,1,...,36.7627,1.82507777,9.34784404,19.03939665,17.97270441,59.29144262,62.39051934,733114886.2389146,0.0408,865036340.98
97,6836,Moonbeam,GLMR,moonbeam,38,2022-01-11T12:55:50.000Z,1000000000,230248311.0,1016966526.0,1,...,-0.5394,0.07479323,-4.31289665,-26.33872006,-5.26702344,16.45949933,-55.23386735,695328108.2329031,0.0387,3019905358.75
98,2586,Synthetix,SNX,synthetix-network-token,233,2018-03-14T00:00:00.000Z,212424133,114841533.01197928,215258834.2449152,1,...,-31.4878,2.28658328,5.12260966,-6.81122112,4.95902968,51.55707639,26.56093187,680751396.3457031,0.0379,1259196227.75
99,2099,ICON,ICX,icon,80,2017-10-27T00:00:00.000Z,,919483096.0,926210351.0,1,...,-20.529,-0.08738574,-0.66202864,-6.07023995,-26.3499595,6.59846316,3.77085954,666920167.4885885,0.0371,671799585.12


In [ ]:
#write the new DF to storage
storage_account_name = config.storage_account_name
storage_account_key = config.storage_account_key
container_name_price = "coinprice"
directory_name = "refined"

container_name_volume = "coinvolume"

df_volume_and_percent_change_csv
df_price_and_supply_csv

dataframe_price_supply = write_to_storage(storage_account_name=storage_account_name, storage_account_key= storage_account_key,\
     container_name= container_name_price, directory_name= directory_name,\
    dataset = df_price_and_supply_csv, file_name= "coinpricerefined.csv")

dataframe_price_supply = write_to_storage(storage_account_name=storage_account_name, storage_account_key= storage_account_key,\
     container_name= container_name_volume, directory_name= directory_name,\
    dataset = df_volume_and_percent_change_csv, file_name= "coinvolumefined.csv")